## Depends on:

A0. Distress Topic Modeling notebook

In [8]:
COMMENTS_FILE = "../working-dir/parsed/classified_comments/sample_total_comments_2018.tsv"
FLATTENED_TREES_FILE = "../working-dir/parsed/classified_comments/flattened_comment_trees_all_2018.tsv"
LIWC_DICT_FILE = "/home/REDACTED/LIWC2015_English.dic"
POST_METADATA_FILE = "../working-dir/parsed/classified_comments/post_metadata_total_2018.tsv"
GRIEF_TOPICS_PATH = "../working-dir/mallet/seeking_topics.txt"

In [1]:
import datetime
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import seaborn as sns
sns.set()
%matplotlib notebook
%load_ext autoreload
%load_ext ipycache

%autoreload 2

np.random.seed(0xB1AB)
# from bert_classifier.utils import *

/opt/anaconda/lib/python3.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/naitian/.local/lib/python3.7/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [2]:
from util import read_labeled_output

In [4]:
(year,
 c,
 cs,
 both) = read_labeled_output(COMMENTS_FILE)

Loading labeled comments from ../working-dir/parsed/classified_comments/sample_total_comments_2018.tsv


/opt/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
def read_trees(fname):
    tree = dict()
    with open(fname, "r") as f:
        for line in f:
            parts = line.strip().split("\t")
            parent = parts[0]
            children = parts[1:]
            tree[parent] = children
    return tree
tree = read_trees(FLATTENED_TREES_FILE)

In [7]:
grief_ids = set(cs.index)
condolence_ids = set(c.index)
both = grief_ids.intersection(condolence_ids)

grief_ids -= both
condolence_ids -= both

grief_dict = cs.to_dict()
condolence_dict = c.to_dict()

no_response = []
no_condolence = []
condolence = []

pairs = []
for i, g_id in enumerate(tree):
    if i % 1_000_000 == 0:
        print(i)
    if g_id in grief_ids:
        c_replies = [c_id for c_id in tree[g_id] if c_id in condolence_ids]
        if len(tree[g_id]) == 0:
            no_response.append((g_id, []))
        elif len(c_replies) > 0:
            condolence.append((g_id, c_replies))
        else:
            no_condolence.append((g_id, tree[g_id]))

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000


In [9]:
grief_topics = pd.read_csv(GRIEF_TOPICS_PATH, sep="\t", names=["index",
                                                               "id",
                                                               "has_diet",
                                                               "has_cursing",
                                                               "who_knows_1",
                                                               "has_social_romantic",
                                                               "has_business",
                                                               "has_games",
                                                               "has_technology",
                                                               "has_family",
                                                               "who_knows_2",
                                                               "has_sports",
                                                               "has_temporal",
                                                               "has_politics",
                                                               "has_career",
                                                               "has_finance",
                                                               "has_nostalgia",
                                                               "has_pet",
                                                               "has_health",
                                                               "has_anecdote",
                                                               "has_mental_health",
                                                               "has_home_moving"], index_col="id")
grief_topics = grief_topics.drop("index", axis=1)

In [ ]:
import genderperformr

def get_gender_scores(usernames):
    gender_scores = np.array([])
    k = 10_000
    for i in range(0, len(usernames), k):
        print(i)
        gender_scores = np.concatenate([gender_scores, genderperformr.predict(usernames.iloc[i:i + k].to_list())[0]])
    gender_scores = gender_scores[:len(usernames)]
    return gender_scores

In [ ]:
# get comment depth...

id_to_depth = dict()

def set_depth(grief_id, depth=0):
    if grief_id in id_to_depth:
        return
    id_to_depth[grief_id] = depth
    for child in tree[grief_id]:
        set_depth(child, depth + 1)

for i, grief_id in enumerate(tree):
    if i % 1_000_000 == 0:
        print(i)
    set_depth(grief_id, 0) 

In [ ]:
import liwc
import re
parse, category_names = liwc.load_token_parser(LIWC_DICT_FILE)
def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

from nltk.tokenize import word_tokenize

def get_ling_accommodation(parent_id, child_id):
    parent_comment = grief_dict["body"][parent_id].lower()
    child_comment = condolence_dict["body"][child_id].lower()
    parent_words = set(word_tokenize(parent_comment))
    child_words = set(word_tokenize(child_comment))
    return len(parent_words.intersection(child_words)) / len(child_words)
        
def get_liwc(body):
    liwc_first_person = 0
    liwc_second_person = 0
    liwc_third_person = 0
    for tok in tokenize(body.lower()):
        for category in parse(tok):
            if category == "i (I)" or category == "we (We)":
                liwc_first_person += 1
            elif category == "you (You)":
                liwc_second_person += 1
            elif category == "shehe (SheHe)" or category == "they (They)":
                liwc_third_person += 1
    return liwc_first_person, liwc_second_person, liwc_third_person

In [ ]:
post_data = pd.read_csv(POST_METADATA_FILE,
                        names=["post_id",
                               "author",
                               "is_self",
                               "created_utc",
                               "score"],
                        index_col="post_id",
                        sep="\t")
post_data["created_utc"] = pd.to_datetime(post_data.created_utc, unit="s", utc=True)

In [ ]:
ids_in_tree = grief_ids.intersection(set(tree))

In [ ]:
# assert False
dataset = cs[["subreddit", "link_id", "body", "seeking_score", "score", "created_utc", "author"]].loc[sample(ids_in_tree, 1_000_000)]

print("Calculating Length")
# Length in space-delimited words
dataset.loc[:, ("length")] = dataset.body.apply(lambda x: len(x.split()))


print("Filter Link IDs")
counts = dataset.link_id.value_counts(ascending=False)
links = set((counts[counts > 50]).index)
dataset.loc[:, "filtered_link_id"] = dataset.loc[:, "link_id"].apply(lambda x: x if x in links else "dummy")

print("Gender Scores")
gender_scores = get_gender_scores(dataset.author)
dataset.loc[:, ("is_male")] = gender_scores < 0.1
dataset.loc[:, ("is_female")] = gender_scores > 0.9

print("Post Features")
dataset.loc[:, "link_id_stripped"] = dataset.loc[:, "link_id"].apply(lambda x: x[3:])
dataset = dataset.join(post_data, on="link_id_stripped", rsuffix="_post")

print("Temporal")
# set dataset timezone
dataset.loc[:, "created_utc"] = dataset.created_utc.dt.tz_convert("US/Eastern")
dataset.loc[:, "created_utc_post"] = dataset.created_utc_post.dt.tz_convert("US/Eastern")
dataset.loc[:, "hour"] = dataset.created_utc.dt.hour
dataset.loc[:, "month"] = dataset.created_utc.dt.month
dataset.loc[:, "day_of_month"] = dataset.created_utc.dt.day
dataset.loc[:, "weekday"] = dataset.created_utc.dt.weekday

print("Topics")
# Before running this, make sure to run topic modeling code below.
dataset = dataset.join(grief_topics)

print("Comment Depth")
dataset.loc[:, "depth"] = np.array([id_to_depth[grief_id] for grief_id in dataset.index])

print("Change Score Type")
dataset.loc[:, "score"] = dataset.loc[:, "score"].astype(np.float)

print("Comment Age in hours")
dataset.loc[:, "comment_age"] = ((dataset.loc[:, "created_utc"] - dataset.loc[:, "created_utc_post"]) / np.timedelta64(1, 'h'))


print("LIWC Features")
dataset.loc[:, "liwc_first_person"], dataset.loc[:, "liwc_second_person"], dataset.loc[:, "liwc_third_person"] = zip(*dataset.body.apply(get_liwc))


print("Response Value")
# Add response values
no_response_ids = set([g_id for g_id, _ in no_response])
condolence_ids = set([g_id for g_id, _ in condolence])
dataset.loc[:, "received_reply"] = [ind not in no_response_ids for ind in dataset.index.values]
dataset.loc[:, "received_condolence"] = [ind in condolence_ids for ind in dataset.index.values]

In [ ]:
dataset.to_csv("../data/regression/distress_input.csv")